In [9]:
!pip install tensorflow roboflow opencv-python matplotlib zipfile36

In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import shutil
from roboflow import Roboflow
from tensorflow.keras.preprocessing import image_dataset_from_directory


In [63]:
rf = Roboflow(api_key="zJBGzdUCic7EvF42E0hk")
project = rf.workspace("uveitis").project("eye-diseases-zncfk")
version = project.version(3)
dataset = version.download("tfrecord")
print(dataset)

loading Roboflow workspace...
loading Roboflow project...


In [59]:
# %% [code]
# Define image dimensions and batch size.
img_height = 224
img_width = 224
batch_size = 32

# Define a parser function to decode the TFRecord examples.
def parse_tfrecord(example_proto):
    # Adjust the feature description based on the TFRecord schema.
    feature_description = {
        "image/encoded": tf.io.FixedLenFeature([], tf.string),
        "image/class/label": tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)
    
    # Decode and resize the image.
    image = tf.image.decode_jpeg(parsed_example["image/encoded"], channels=3)
    image = tf.image.resize(image, [img_height, img_width])
    
    # Cast the label to an integer.
    label = tf.cast(parsed_example["image/class/label"], tf.int32)
    return image, label

# List TFRecord files.
# (Assumes your directory contains files like "train*.tfrecord" and "valid*.tfrecord")
train_pattern = os.path.join(dataset.location, "train*.tfrecord")
val_pattern   = os.path.join(dataset.location, "valid*.tfrecord")

train_files = tf.io.gfile.glob(train_pattern)
val_files   = tf.io.gfile.glob(val_pattern)

print("Training files:", train_files)
print("Validation files:", val_files)

# Create TFRecordDataset objects.
train_ds = tf.data.TFRecordDataset(train_files)
val_ds   = tf.data.TFRecordDataset(val_files)

# Parse the datasets.
train_ds = train_ds.map(parse_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_ds   = val_ds.map(parse_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle, batch, and prefetch the datasets.
train_ds = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds   = val_ds.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# (Optional) Load class names from a file if provided by Roboflow.
classes_file = os.path.join(dataset.location, "classes.txt")
if os.path.exists(classes_file):
    with open(classes_file, "r") as f:
        class_names = f.read().splitlines()
    print("Detected classes:", class_names)
else:
    # If there is no classes.txt, set class_names manually (update as needed)
    class_names = ["class0", "class1"]
    print("Using default class names:", class_names)

num_classes = len(class_names)


Training files: []
Validation files: []
Using default class names: ['class0', 'class1']


In [61]:
# %% [code]
# Build a simple CNN model.
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(img_height, img_width, 3)),
    tf.keras.layers.Rescaling(1.0/255),  # Normalize pixel values to [0, 1].
    
    # Convolutional layers.
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Softmax for multi-class classification.
])

model.summary()

# Compile the model.
# Since our labels are integer encoded, we use sparse categorical crossentropy.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model.
epochs = 10  # You can adjust the number of epochs as needed.
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

# (Optional) Save the trained model.
model.save("eye_diseases_model.h5")


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,218 (42.61 MB)

 Trainable params: 11,169,218 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.


ValueError: math domain error